# Big city music by Yandex.Music

Comparison of Moscow and St. Petersburg is surrounded by myths. For example:
 * Moscow is a metropolis subject to the rigid rhythm of the working week;
 * Petersburg is a cultural capital, with its own tastes.
    
Based on Yandex.Music data, we will compare the behavior of users in the two capitals.

**Purpose of the research** — 3 hypotheses need to be tested:
1. User activity depends on the day of the week. Moreover, in Moscow and St. Petersburg this manifests itself in different ways.
2. On Monday morning, certain genres dominate in Moscow, while others dominate in St. Petersburg. Similarly, Friday evenings are dominated by different genres, depending on the city.
3. Moscow and St. Petersburg prefer different genres of music. In Moscow, they listen to pop music more often, in St. Petersburg - Russian rap.

**Research progress:**

  1. Data review.
  2. Data preprocessing.
  3. Hypothesis testing.


## Data review

Let's make the first view of the data Yandex.Music

In [1]:
# Import the pandas library, immediately renaming it to "pd" for convenience
import pandas as pd

In [2]:
# Read the "yandex_music.csv" file  and save it in the "df" variable:
df = pd.read_csv('yandex_music.csv')

In [3]:
# Let's display the first ten rows of the table, and read the general information about the dataset:
display(df.head(10))
df.info()

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


So the table has seven columns. The data type in all columns is `object`.

According to the data documentation:
* `userID` - user ID;
* `Track` — track name;
* `artist` — artist name;
* `genre` — genre name;
* `City` - user's city;
* `time` - start time of listening;
* `Day` is the day of the week.

There are three style violations in the column headings:
1. Lowercase letters are combined with uppercase.
2. There are gaps.
3. It is better to write the names in "snake case" (user_id)

The number of values in the columns varies. This means there are missing values in the data.

**Conclusion**

Each line of the table contains data about the track you have listened to. Some of the columns describe the composition itself: `title`, `artist` and `genre`. The rest of the data tells about the user: what city he is from, when he listened to music.

Preliminarily, it can be argued that there is enough data to test hypotheses. But there are gaps in the data, and discrepancies in the names of the columns with good style.

To move forward, we need to fix problems in the data.

## Data preprocessing:

### Column style:

In [4]:
# Let's see the column names:
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Let's bring the names in line with good style:
* we will write the names in the “snake_case”,
* make all characters lowercase,
* eliminate spaces.
___
* `'  userID'` → `'user_id'`;
* `'Track'` → `'track'`;
* `'  City  '` → `'city'`;
* `'Day'` → `'day'`.

In [5]:
df_filtered = df.rename(columns = {'  userID': 'user_id', 'Track': 'track', '  City  ': 'city','Day': 'day'})
df_filtered.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Missing values:

In [6]:
# Let's see how many missing values are in the table:
df_filtered.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Not all missing values affect the study. So in `track` and `artist` gaps are not important for your work. It suffices to replace them with explicit notation.

But gaps in the `genre` can interfere with the comparison of musical tastes in Moscow and St. Petersburg. In practice, it would be correct to determine the cause of the gaps and restore the data. But in this case, there is no such possibility. Have to:

- fill in these gaps with explicit notation,
- estimate how much they will damage the calculations.



In [7]:
# We will iterate over the column names in a loop and replace missing values with 'unknown':
columns_to_replace = ['track', 'artist', 'genre']
for col in columns_to_replace:
    df_filtered[col] = df_filtered[col].fillna('unknown')
    
df_filtered.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Duplicates:

In [8]:
# Let's count the explicit duplicates:
df_filtered.duplicated().sum()

3826

In [9]:
# Removing apparent duplicates (deleting old indexes and creating new ones):
df_filtered = df_filtered.drop_duplicates().reset_index(drop= True)

df_filtered.duplicated().sum()

0

Now let's get rid of the implicit duplicates in the `genre` column. For example, the name of the same genre can be spelled slightly differently. Such errors will also affect the result of the study.

Let's display a list of unique `genre` names, sorted alphabetically. For this:
* extract the desired dataframe column,
* apply sorting method to it,
* for a sorted column, call a method that will return unique values from the column.

In [10]:
df_filtered['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Found implicit duplicates of the name `hiphop`. :

**hip*,
**hop*,
**hip-hop*.

To clear the table of them, we write the `replace_wrong_genres()` function with two parameters:
* `wrong_genres` - list of duplicates,
* `correct_genre` is a string with the correct value.

The function should correct the `genre` column in the `df` table: replace each value from the `wrong_genres` list with a value from `correct_genre`.

In [11]:
# Create and use a function:
def replace_wrong_genres(wrong_genres, correct_genre):
    for genre in wrong_genres:
        df_filtered['genre'] = df_filtered['genre'].replace(genre, correct_genre)

wrong_genres = ['hip', 'hop','hip-hop']
correct_genre = 'hiphop'
replace_wrong_genres(wrong_genres, correct_genre)

In [12]:
df_filtered['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

**Conclusions:**

Preprocessing found three problems in the data:

- headline style violations,
- missing values,
- duplicates - explicit and implicit.

We've fixed the headers to make the table easier to work with. Without duplicates, the study will become more accurate.

We have replaced missing values with `'unknown'`. It remains to be seen whether the gaps in the `genre` column will harm the study.

Now we can move on to hypothesis testing.

## Testing hypotheses

### Comparison of user behavior in two capitals

The first hypothesis states that users listen to music differently in Moscow and St. Petersburg. Let's check this assumption on the data on three days of the week - Monday, Wednesday and Friday. For this:

* Separate users of Moscow and St. Petersburg
* Compare how many tracks each group of users listened to on Monday, Wednesday and Friday.

Let's do each of the calculations separately.
Let's evaluate the activity of users in each city. Let's group the data by city and count the plays in each group.

In [13]:
# Let's calculate the number of plays in each city:
df_filtered.groupby('city')['user_id'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: user_id, dtype: int64

There are more auditions in Moscow than in St. Petersburg. It does not follow from this that Moscow users listen to music more often. There are simply more users in Moscow.

Let's group the data by day of the week and count the plays on Monday, Wednesday, and Friday. Let's take into account that the data contains information only about listening only for these days.

In [14]:
# Count the number of plays on each of the three days:
df_filtered.groupby('day')['user_id'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: user_id, dtype: int64

On average, users from the two cities are less active on Wednesdays. But the picture may change if we consider each city separately.

We see how the grouping works by city and by day of the week. Now let's write a function that combines these two calculations.

Let's create a `number_tracks()` function that will count the plays for a given day and city. She needs two parameters:
* day of the week,
* city name.

In the function, we save the rows of the source table into a variable, which have the value:
  * in the `day` column is equal to the `day` parameter,
  * in the `city` column is equal to the `city` parameter.

To do this, apply sequential filtering with logical indexing.

Then we count the values in the `user_id` column of the resulting table. The result will be stored in a new variable. Let's return this variable from the function.

In [15]:
# Create and use a function:
def number_tracks(day,city):
    track_list = df_filtered[(df_filtered['day'] == day) & (df_filtered['city'] == city)]
    track_list_count = track_list['user_id'].count()
    return track_list_count


moscow_monday_list = number_tracks('Monday','Moscow')
moscow_wednesday_list = number_tracks('Wednesday','Moscow')
moscow_friday_list = number_tracks('Friday','Moscow')

spb_monday_list = number_tracks('Monday','Saint-Petersburg')
spb_wednesday_list = number_tracks('Wednesday','Saint-Petersburg')
spb_friday_list = number_tracks('Friday','Saint-Petersburg')


In [16]:
# Let's create tables with results:
columns = ['city', 'monday', 'wednesday', 'friday']
data_list_day = [['Moskow', moscow_monday_list, moscow_wednesday_list , moscow_friday_list], 
                 ['Saint-Petersburg', spb_monday_list, spb_wednesday_list, spb_friday_list]]
df_day_list = pd.DataFrame(data = data_list_day, columns = columns)
display(df_day_list)

,city,monday,wednesday,friday
0,Moskow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Conclusions:**

The data shows the difference in user behavior:

- In Moscow, the peak of listening falls on Monday and Friday, and on Wednesday there is a noticeable decline.
- In St. Petersburg, on the contrary, they listen to music more on Wednesdays. Activity on Monday and Friday here is almost equally inferior to Wednesday.

So the data support the first hypothesis.

### Music at the beginning and end of the week:

According to the second hypothesis, on Monday morning certain genres predominate in Moscow, while others dominate in St. Petersburg. Similarly, Friday evenings are dominated by different genres, depending on the city. 

In [17]:
# Let's divide the table with data into two by cities:
moscow_general = df_filtered[df_filtered['city'] == 'Moscow']
spb_general = df_filtered[df_filtered['city'] == 'Saint-Petersburg']

Let's create a `genre_weekday()` function with four parameters:
* table (dataframe) with data,
* day of the week,
* start timestamp in 'hh:mm' format,
* last timestamp in 'hh:mm' format.

The function should return information about the top 10 genres of those tracks that were listened to on the specified day, in the interval between two timestamps.

In [18]:
def genre_weekday(table, day, time1, time2):
    genre_df = table[(table['day'] == day) & (table['time'] > time1) & (table['time'] < time2)]
    genre_df_count = genre_df.groupby('genre')['user_id'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending = False)
    return genre_df_sorted.head(10)

Let's compare the results of the `genre_weekday()` function for Moscow and St. Petersburg on Monday morning (from 7:00 to 11:00) and Friday evening (from 17:00 to 23:00):

In [19]:
print("Moscow Monday 07:00-11:00:")
print(genre_weekday(moscow_general, 'Monday', '07:00', '11:00'))
print()
print("SPB Monday 07:00-11:00:")
print(genre_weekday(spb_general, 'Monday', '07:00', '11:00'))

Moscow Monday 07:00-11:00:
genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: user_id, dtype: int64

SPB Monday 07:00-11:00:
genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: user_id, dtype: int64


In [20]:
print("Moscow Friday 17:00-23:00:")
print(genre_weekday(moscow_general, 'Friday', '17:00', '23:00'))
print()
print("SPB Friday 17:00-23:00:")
print(genre_weekday(spb_general, 'Friday', '17:00', '23:00'))

Moscow Friday 17:00-23:00:
genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: user_id, dtype: int64

SPB Friday 17:00-23:00:
genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: user_id, dtype: int64


**Conclusions:**

If we compare the top 10 genres on Monday morning, we can draw the following conclusions:

1. In Moscow and St. Petersburg they listen to similar music. The only difference is that the Moscow rating includes the “world” genre, while the St. Petersburg rating includes jazz and classical.

2. There were so many missing values ​​in Moscow that the value `'unknown'` took tenth place among the most popular genres. This means that missing values ​​occupy a significant share in the data and threaten the reliability of the study.

Friday night does not change this picture. Some genres rise a little higher, others go down, but overall the top 10 stays the same.

Thus, the second hypothesis was only partially confirmed:
* Users listen to similar music at the beginning of the week and at the end.
* The difference between Moscow and St. Petersburg is not very pronounced. In Moscow, they listen to Russian popular music more often, in St. Petersburg - jazz.

However, gaps in the data cast doubt on this result. There are so many of them in Moscow that the top 10 ranking could look different if it were not for the lost genre data.

### Genre preferences in Moscow and St. Petersburg:

Hypothesis: St. Petersburg is the capital of rap, the music of this genre is listened to more often than in Moscow. And Moscow is a city of contrasts, which, nevertheless, is dominated by pop music.

Let's group the table of each city by genre and count the listens to tracks of each genre using the `count()` method and output the first 10 lines. 

In [21]:
print('Moscow popular:')
print(moscow_general.groupby('genre')['genre'].count().sort_values(ascending =False).head(10))

Moscow popular:
genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64


In [22]:
print('SPB popular:')
print(spb_general.groupby('genre')['genre'].count().sort_values(ascending =False).head(10))

SPB popular:
genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64


**Conclusions**

The hypothesis was partially confirmed:
* Pop music is the most popular genre in Moscow, as the hypothesis suggested. Moreover, in the top 10 genres there is a close genre - Russian popular music.
* Contrary to expectations, rap is equally popular in Moscow and St. Petersburg.

## Results of the research

We tested three hypotheses and found:

1. The day of the week has a different effect on the activity of users in Moscow and St. Petersburg.

The first hypothesis was fully confirmed.

2. Musical preferences do not change much during the week - be it Moscow or St. Petersburg. Small differences are noticeable at the beginning of the week, on Mondays:
* in Moscow they listen to music of the “world” genre,
* in St. Petersburg - jazz and classical music.

Thus, the second hypothesis was only partly confirmed. This result could have been different were it not for gaps in the data.

3. The tastes of users of Moscow and St. Petersburg have more in common than differences. Contrary to expectations, genre preferences in St. Petersburg resemble those in Moscow.

The third hypothesis was not confirmed. If there are differences in preferences, they are invisible to the bulk of users.

*For the study, data from "Yandex Practicum" were used.